<a href="https://colab.research.google.com/github/nikogarro/myapp/blob/main/myapp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inspiration
https://www.youtube.com/watch?v=1TYyX8soQ8M&list=PLhYNDxVvF4oXa9ihs8WCzEriZsCF3Pp7A&index=3

https://developer.spotify.com/dashboard/login

In [ ]:
!pip install pyngrok==4.1.1
!pip install spotipy
!pip install flask_ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from flask import Flask, request, url_for, session, redirect
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from flask_ngrok import run_with_ngrok
from pyngrok import ngrok
import time
import pandas as pd

In [ ]:
!ngrok authtoken 2JRmZO5cGImCTi4iig49ejRQ6ab_6bUBXYP5RL4esBgBKjPqm

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
from os import access
myapp = Flask(__name__)

#run_with_ngrok(myapp)

#secret key to sign our session cookie 
myapp.secret_key = '1234567890NG'

#set session cookie name 
myapp.config['SESSION_COOKIE_NAME'] = 'Niko Cookie'
#key to get information in a session dictionary
TOKEN_INFO = "token_info"

#setting up an end point or route
#using the login() function that automatically 
#log us in spotify when in index page
@myapp.route('/')
def login():
    sp_oauth = create_spotify_oauth()
    auth_url = sp_oauth.get_authorize_url()
    print(auth_url)
    return redirect(auth_url)

@myapp.route('/redirect')
def redirectPage():
    sp_oauth = create_spotify_oauth()
    # clearing any previous session states
    session.clear()
    code = request.args.get('code')
    #getting token information
    token_info = sp_oauth.get_access_token(code)
    #saving token info in the session
    session[TOKEN_INFO] = token_info
    return redirect("/getTracks")

@myapp.route('/logout')
def logout():
    for key in list(session.keys()):
        session.pop(key)
    return redirect('/')

#get the tracks end point or route
@myapp.route('/getTracks')
def get_all_tracks():
    session['token_info'], authorized = get_token()
    session.modified = True
    if not authorized:
        return redirect('/')
    sp = spotipy.Spotify(auth=session.get('token_info').get('access_token'))
    results = []
    iter = 0
    while True:
        offset = iter * 50
        iter += 1
        curGroup = sp.current_user_saved_tracks(limit=50, offset=offset)['items']
        for idx, item in enumerate(curGroup):
            track = item['track']
            val = track['name'] + " - " + track['artists'][0]['name']
            results += [val]
        if (len(curGroup) < 50):
            break
    
    df = pd.DataFrame(results, columns=["song names"]) 
    df.to_csv('songs.csv', index=False)
    return "done"

#check if the token is expired and get us a fresh one
def get_token():
    token_valid = False
    token_info = session.get("token_info", {})

    # Checking if the session already has a token stored
    if not (session.get('token_info', False)):
        token_valid = False
        return token_info, token_valid

    # Checking if token has expired
    now = int(time.time())
    is_token_expired = session.get('token_info').get('expires_at') - now < 60

    # Refreshing token if it has expired
    if (is_token_expired):
        sp_oauth = create_spotify_oauth()
        token_info = sp_oauth.refresh_access_token(session.get('token_info').get('refresh_token'))

    token_valid = True
    return token_info, token_valid

def create_spotify_oauth():
    return SpotifyOAuth(
        client_id='a558f3ccbafc4c52be2c1157b3452722',
        client_secret='e56013de61c14f9f91f646fd9eeb421c',
        redirect_uri=url_for('redirectPage', _external=True),
        scope='user-library-read')
    
#myapp.run()
# Open a HTTP tunnel on the default port 80
public_url = ngrok.connect(port = '5000')
ssh_url = ngrok.connect(22,'tcp')
public_url

'http://8132-35-243-230-223.ngrok.io'

## Get the one time url from ngrok and register it at the spotify api Developer Dashboard with an /redirect

In [ ]:
# Open a HTTP tunnel on the default port 80
public_url = ngrok.connect(port = '5000')
ssh_url = ngrok.connect(22,'tcp')
public_url

## Start the flask server in the one time use browser

In [ ]:
myapp.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
INFO:werkzeug:127.0.0.1 - - [27/Dec/2022 14:25:34] "GET / HTTP/1.1" 302 -


https://accounts.spotify.com/authorize?client_id=a558f3ccbafc4c52be2c1157b3452722&response_type=code&redirect_uri=http%3A%2F%2F8132-35-243-230-223.ngrok.io%2Fredirect&scope=user-library-read


<ipython-input-6-15f8694bba99>:31: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  token_info = sp_oauth.get_access_token(code)
INFO:werkzeug:127.0.0.1 - - [27/Dec/2022 14:25:34] "GET /redirect?code=AQBlz8DoOh55haGOs-LFZ4zb9AMtQ5FwKZNtb-B-IWY8SI7tewBOOwRNIi4DRO-NKBbJO8Z7oOxBfDJy7Wh_OmJbLahe2nR21p24YGUyHsa2Mt6L0AoRCDzXjWONdeCU_c7JTTajeZ5mOftl3gqmT57alZQ0HOTmMtl5c3OG0aJjC4z6QLu8_2_n2w8B-hc79oyTMqe-C-VV4N-dr_gTXmx22g HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [27/Dec/2022 14:25:35] "GET /getTracks HTTP/1.1" 200 -


In [ ]:
#Terminate ngrok processes, only one at a time allowed also fixes HTTP tunnel error 
ngrok.kill()

## Aquire song ids/urls with BeautifulSoup and Download them with youtube_dl

Copy the following code in a `.py` file and save it then run the file through the **command line** example `cd C://path` of the directory of the file and afterwards do `python filename.py` make sure the CSV file `songs.csv` is in the same directory as the python file.The first time you run the file `Chromium` will be downloaded which is the browser the scrapper will work on in the background.

In [15]:
#downloading songs from youtube
from bs4 import BeautifulSoup
from requests_html import HTMLSession
from pathlib import Path
import youtube_dl
import requests
import pandas
import os

def DownloadVideosFromTitles(los):
	ids = []
	for index, item in enumerate(los):
		vid_id = ScrapeVidId(item)
		ids += [vid_id]
	print("Downloading songs")
	DownloadVideosFromIds(ids)


def DownloadVideosFromIds(lov):
	SAVE_PATH = str(os.path.join(Path.home(), "Downloads/songs"))
	try:
		os.mkdir(SAVE_PATH)
	except:
		print("download folder exists")
	ydl_opts = {
    	'format': 'bestaudio/best',
   	'''	'postprocessors': [{
        		'key': 'FFmpegExtractAudio',
        		'preferredcodec': 'mp3',
        		'preferredquality': '192',
    		}],'''
		'outtmpl': SAVE_PATH + '/%(title)s.%(ext)s',
	}
	with youtube_dl.YoutubeDL(ydl_opts) as ydl:
	    ydl.download(lov)

async def ScrapeVidId(query):
    print ("Getting video id for: ", query)
    print("Lenght of the seached song used to help debugging",len(query))
    BASIC="http://www.youtube.com/results?search_query=" 
    query = query.replace(" ", "+")
    URL = (BASIC + query)
    print("The URL is: ", URL)
    URL = URL.replace(' ', '+')

    print("The URL after replacing the spaces with + is: ", URL)
    session = HTMLSession()
    response = session.get(URL)
    #
    response.html.render(sleep=1, timeout=900000)
    soup = BeautifulSoup(response.html.html, "html.parser")
    print("The soup is: ", soup)
    results = soup.find("a", id="video-title")
    print('the results variable value is:', results)
    return results['href'].split('/watch?v=')[1]

def __main__():
    data = pandas.read_csv('songs.csv')
    songlist = data['song names'].tolist()
    print(songlist)
    print("Found ", len(songlist), " songs!")
    DownloadVideosFromTitles(songlist[0:1])
__main__()

['Fall Apart - Tearz', 'I Chose Violence - iamjakehill', 'Enemy (with JID) - from the series Arcane League of Legends - Imagine Dragons', 'The Sound - Aero Chord', 'Hold On & Believe (feat. The Federal Empire) - Martin Garrix', 'Sun Is Never Going Down (feat. Dawn Golden) - Martin Garrix', 'Lamp (feat. Nujabes) - haruka nakamura', 'Spaceships - AREA21', 'Spotless - Martin Garrix', 'How Many - W&W', 'Rave After Rave - W&W', "Don't Let Me Down - Illenium Remix - The Chainsmokers", 'Hymn of Reflection - KSHMR', 'Touch (EP Version) - KSHMR', 'Dadima - KSHMR', 'Wildcard (EP Version) - KSHMR', 'Sleepwalk - KSHMR', 'Jungle Whistle - KSHMR', 'Lost Words - Radio Edit - Julian Jordan', 'Stardust - Jay Hardway', 'Electric Elephants - Jay Hardway', 'Louder - MOTi', 'El Mariachi - Bassjackers', 'Into The Fire - VINAI', 'Sing Me to Sleep - Marshmello Remix - Alan Walker', "Don't Let Me Down - W&W Remix - The Chainsmokers", 'Pirates - Crystal Lake', 'Forbidden Voices - Martin Garrix', '4U - Aero Chor

TypeError: ignored

## Method 2 using selenium


In [ ]:
!apt-get update
!pip install selenium
!apt install firefox-geckodriver
!cp /usr/lib/geckodriver/usr/bin
!cp /usr/lib/firefox/usr/bin

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done
Looking in indexes: https://pypi.org/simple, https:

In [ ]:
#downloading songs from youtube
from bs4 import BeautifulSoup
from requests_html import HTMLSession
from pathlib import Path
import youtube_dl
import requests
import pandas
import os
from selenium import webdriver
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC


def DownloadVideosFromTitles(los):
	ids = []
	for index, item in enumerate(los):
		vid_id = ScrapeVidId(item)
		ids += [vid_id]
	print("Downloading songs")
	DownloadVideosFromIds(ids)


def DownloadVideosFromIds(lov):
	SAVE_PATH = str(os.path.join(Path.home(), "Downloads/songs"))
	try:
		os.mkdir(SAVE_PATH)
	except:
		print("download folder exists")
	ydl_opts = {
    	'format': 'bestaudio/best',
   		'postprocessors': [{
        		'key': 'FFmpegExtractAudio',
        		'preferredcodec': 'mp3',
        		'preferredquality': '192',
    		}],
		'outtmpl': SAVE_PATH + '/%(title)s.%(ext)s',
	}
	with youtube_dl.YoutubeDL(ydl_opts) as ydl:
	    ydl.download(lov)

def ScrapeVidId(query):
    print ("Getting video id for: ", query)
    print("Lenght of the seached song used to help debugging",len(query))
    BASIC="http://www.youtube.com/results?search_query=" 
    query = query.replace(" ", "+")
    URL = (BASIC + query)
    print("The URL is: ", URL)
    URL = URL.replace(' ', '+')
    print("The URL after replacing the spaces with + is: ", URL)

def __main__():
    data = pandas.read_csv('/songs.csv')
    songlist = data['song names'].tolist()
    print(songlist)
    print("Found ", len(songlist), " songs!")
    DownloadVideosFromTitles(songlist[0:1])
__main__()

['Fall Apart - Tearz', 'I Chose Violence - iamjakehill', 'Enemy (with JID) - from the series Arcane League of Legends - Imagine Dragons', 'The Sound - Aero Chord', 'Hold On & Believe (feat. The Federal Empire) - Martin Garrix', 'Sun Is Never Going Down (feat. Dawn Golden) - Martin Garrix', 'Lamp (feat. Nujabes) - haruka nakamura', 'Spaceships - AREA21', 'Spotless - Martin Garrix', 'How Many - W&W', 'Rave After Rave - W&W', "Don't Let Me Down - Illenium Remix - The Chainsmokers", 'Hymn of Reflection - KSHMR', 'Touch (EP Version) - KSHMR', 'Dadima - KSHMR', 'Wildcard (EP Version) - KSHMR', 'Sleepwalk - KSHMR', 'Jungle Whistle - KSHMR', 'Lost Words - Radio Edit - Julian Jordan', 'Stardust - Jay Hardway', 'Electric Elephants - Jay Hardway', 'Louder - MOTi', 'El Mariachi - Bassjackers', 'Into The Fire - VINAI', 'Sing Me to Sleep - Marshmello Remix - Alan Walker', "Don't Let Me Down - W&W Remix - The Chainsmokers", 'Pirates - Crystal Lake', 'Forbidden Voices - Martin Garrix', '4U - Aero Chor

<ipython-input-186-cf61e1bcdd61>:56: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Firefox(options=options, executable_path='/usr/bin/geckodriver')


WebDriverException: ignored